In [ ]:
# Enable autoreload for hot reloading modules
%load_ext autoreload
%autoreload 2

In [ ]:
import sys

# Some how videoseal is added to sys.path at index 0, which causes conflicts
# Remove it to avoid import issues
if "videoseal" in sys.path[0]:
    del sys.path[0]


sys.path.insert(0, "/checkpoint/avseal/tuantran/env_srcs/editguard/editguard/code")  

sys.path

['/checkpoint/avseal/tuantran/env_srcs/editguard/editguard/code',
 '/checkpoint/avseal/tuantran/envs/editguard/lib/python310.zip',
 '/checkpoint/avseal/tuantran/envs/editguard/lib/python3.10',
 '/checkpoint/avseal/tuantran/envs/editguard/lib/python3.10/lib-dynload',
 '',
 '/checkpoint/avseal/tuantran/envs/editguard/lib/python3.10/site-packages']

## Loading eager mode

In [3]:
import torch

from collections import OrderedDict

import models.networks as networks
from data.test_dataset_td import imageTestDataset as D
import options.options as option
from models import create_model

device = "cuda"
opt = option.parse("/checkpoint/avseal/tuantran/env_srcs/editguard/editguard/code/options/test_editguard.yml", is_train=False)
opt = option.dict_to_nonedict(opt)

opt['datasets']['TD']['data_path'] = 'dataset/valAGE-Set'
opt['datasets']['TD']['txt_path'] = 'dataset/sep_testlist.txt'

# Load datasetb
dataset = D(opt['datasets']['TD'])
dataloader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=False, num_workers=1, pin_memory=True)

model = create_model(opt)
model.load_test("/checkpoint/avseal/tuantran/omnisealbench/checkpoints/editguard/clean.pth")
    

export CUDA_VISIBLE_DEVICES=0


/checkpoint/avseal/tuantran/envs/editguard/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


SEAttention
SEAttention
SEAttention
SEAttention
SEAttention
SEAttention
SEAttention
SEAttention
SEAttention
SEAttention
SEAttention
SEAttention
SEAttention
SEAttention
SEAttention
SEAttention


Loading pipeline components...:   0%|                                                                                                                                                                                                                                     | 0/5 [00:00<?, ?it/s]An error occurred while trying to fetch /checkpoint/avseal/cache/huggingface/hub/models--philschmid--stable-diffusion-2-inpainting-endpoint/snapshots/13a0c9d3340289950648a93a0432f2db27531b99/vae: Error no file named diffusion_pytorch_model.safetensors found in directory /checkpoint/avseal/cache/huggingface/hub/models--philschmid--stable-diffusion-2-inpainting-endpoint/snapshots/13a0c9d3340289950648a93a0432f2db27531b99/vae.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
Loading pipeline components...:  60%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                          

In [4]:
from utils import util

for image_id, val_data in enumerate(dataloader):
    model.feed_data(val_data)
    print(image_id)
    model.test(image_id, masksrc="dataset/valAGE-Set-Mask/")
    
    visuals = model.get_current_visuals()
    
    t_step = visuals['SR'].shape[0]
    n = len(visuals['SR_h'])

    a = visuals['recmessage'][0]
    b = visuals['message'][0]

    bitrecord = util.decoded_message_error_rate_batch(a, b)
    print(bitrecord)
    
    break

0


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:05<00:00,  9.99it/s]


0.0


## Loading jit mode

In [28]:
for image_id, val_data in enumerate(dataloader):
    print(val_data["GT"].shape)
    
    break

torch.Size([1, 1, 3, 512, 512])


In [26]:
from models.modules.common import DWT,IWT

dwt = DWT()
iwt = IWT()

val_data_1 = dwt(val_data["GT"].squeeze(0))
val_data_2 = iwt(val_data_1)

torch.testing.assert_close(val_data["GT"].squeeze(0).cpu(), val_data_2.cpu()), "DWT and IWT are not consistent"

(None, 'DWT and IWT are not consistent')

In [ ]:
import numpy as np
from utils import util

bitencoder = torch.jit.load("/checkpoint/avseal/models/baselines-watermarking/editguard_encoder.pt")
bitencoder = bitencoder.eval().to(device)

bitdecoder = torch.jit.load("/checkpoint/avseal/models/baselines-watermarking/editguard_decoder.pt")
bitdecoder = bitdecoder.eval().to(device)

def dwt_init(x):

    x01 = x[:, :, 0::2, :] / 2
    x02 = x[:, :, 1::2, :] / 2
    x1 = x01[:, :, :, 0::2]
    x2 = x02[:, :, :, 0::2]
    x3 = x01[:, :, :, 1::2]
    x4 = x02[:, :, :, 1::2]
    x_LL = x1 + x2 + x3 + x4
    x_HL = -x1 - x2 + x3 + x4
    x_LH = -x1 + x2 - x3 + x4
    x_HH = x1 - x2 - x3 + x4

    return torch.cat((x_LL, x_HL, x_LH, x_HH), 1)

bsz = 1
messagenp = np.random.choice([-0.5, 0.5], (bsz, 64))
msg = torch.Tensor(messagenp).to(device)

for image_id, val_data in enumerate(dataloader):
    
    real_H = val_data["GT"].to(device)
    b, t, c, h, w = real_H.shape
    center = t // 2

    host = real_H[:,center:center + 1].squeeze(0)
    print(host.shape)
    output_y, _ = bitencoder(host, msg)
    output_y = torch.clamp(output_y, 0, 1)
    output_y = (output_y * 255.).round() / 255.
    
    rec_msg = bitdecoder(output_y)
    msg = torch.clamp(msg, -0.5, 0.5)
    rec_msg = torch.clamp(rec_msg, -0.5, 0.5)
    
    bitrecord = util.decoded_message_error_rate_batch(msg, rec_msg)
    print(bitrecord)
    
    break

torch.Size([1, 1, 3, 512, 512])


RuntimeError: The following operation failed in the TorchScript interpreter.
Traceback of TorchScript, serialized code (most recent call last):
  File "code/__torch__/models/bitnetwork/Encoder_U.py", line 34, in forward
    _0 = __torch__.torch.nn.functional.___torch_mangle_145.interpolate
    conv1 = self.conv1
    d0 = (conv1).forward(x, )
          ~~~~~~~~~~~~~~ <--- HERE
    down1 = self.down1
    d1 = (down1).forward(d0, )
  File "code/__torch__/models/bitnetwork/ConvBlock.py", line 10, in forward
    x: Tensor) -> Tensor:
    layers = self.layers
    return (layers).forward(x, )
            ~~~~~~~~~~~~~~~ <--- HERE
class ConvINRelu(Module):
  __parameters__ = []
  File "code/__torch__/torch/nn/modules/container/___torch_mangle_3.py", line 12, in forward
    _0 = getattr(self, "0")
    _1 = getattr(self, "1")
    input0 = (_0).forward(input, )
              ~~~~~~~~~~~ <--- HERE
    return (_1).forward(input0, )
  def __len__(self: __torch__.torch.nn.modules.container.___torch_m

In [12]:
# model.netG.module.bitencoder

In [ ]:
bitdecoder = model.netG.bitencoder
jit_decoder = torch.jit.script(bitdecoder)  # example_inputs=(torch.randn(1

torch.jit.save(jit_decoder, "/checkpoint/avseal/models/baselines-watermarking/editguard_encoder.pt")

AttributeError: 'Model_VSN' object has no attribute 'bitdecoder'